In [37]:
import pandas as pd
import numpy as np
import re

In [14]:
weibo_m = pd.read_csv('./data/Weibo_M.csv')
weibo = pd.read_csv('./data/Weibo.csv')
wechat = pd.read_csv('./data/Wechat.csv')
news_forum = pd.read_csv('./data/News_forum.csv')

In [15]:
weibo_m = weibo_m[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
weibo = weibo[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
wechat = wechat[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
news_forum = news_forum[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')

In [16]:
weibo_m.columns = ['CONTENT','AREAS AFFECTED']
weibo.columns = ['CONTENT','AREAS AFFECTED']

In [41]:
data = pd.concat([weibo_m,weibo,wechat])

In [42]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('life', 'Life')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('sponsored', 'Sponsored')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('news', 'News')

In [43]:
data['AREAS AFFECTED'].unique()

array(['Life Comprehend', 'Products/Service', 'Health Information',
       'Ads/Marketing', 'Customer Service', 'Irrelevant Ads',
       'Charity Events', 'Survey/Questions', 'Contact Information', 'Fund',
       'Corporate Brands', 'Sponsored Events', 'Volunteering Activity',
       'Products', 'Website Issues', 'Corporate News', 'Recruitment',
       'Agent Issues', 'Employment', 'Stocks&Earnings',
       'General Mentioned', 'Employee Information'], dtype=object)

In [25]:
for x in data['AREAS AFFECTED'].unique():
    if(len(data[data['AREAS AFFECTED'] == x]['CONTENT'])>2000):
        np.savetxt('./data/'+ x.replace('/',' ') +'.utf8',\
           data[data['AREAS AFFECTED'] == x]['CONTENT'][:2000].values, fmt='%s')
    else:
        np.savetxt('./data/'+ x.replace('/',' ') +'.utf8',\
               data[data['AREAS AFFECTED'] == x]['CONTENT'].values, fmt='%s')

In [45]:
data.shape

(12326, 2)

In [44]:
for x in data['AREAS AFFECTED'].unique():
    np.savetxt('./data/'+ x.replace('/',' ') +'.utf8',\
                   data[data['AREAS AFFECTED'] == x]['CONTENT'].values, fmt='%s')